# This is for wwall subroutine in vacuum_vac.f , and has not completed yet.

In [1]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
using JPEC, Plots

  Activating project at `~/Desktop/code/GPEC_hackaton/JPEC`


In [2]:
const pye = π ;

In [3]:


# --- Helper Functions (replacing missing Fortran subroutines) ---

"""
    errmes(io::IO, subprog_name::String)

Placeholder for the `errmes` subroutine.
Prints an error message to the specified IO stream.
"""
function errmes(io::IO, subprog_name::String)
    @error "$subprog_name"
end


"""
    bounds!(x1::Vector{Float64}, z1::Vector{Float64}, n1::Int, n2::Int,
            xmin_ref::Ref{Float64}, xmax_ref::Ref{Float64},
            zmin_ref::Ref{Float64}, zmax_ref::Ref{Float64})

Calculates the minimum and maximum X and Z coordinates within a specified range
of two input vectors. This function modifies the provided `Ref` arguments in-place.

# Arguments
- `x1::Vector{Float64}`: Input vector for X coordinates.
- `z1::Vector{Float64}`: Input vector for Z coordinates.
- `n1::Int`: Starting index for the range (Fortran 1-based index).
- `n2::Int`: Ending index for the range (Fortran 1-based index).
- `xmin_ref::Ref{Float64}`: Reference to store the calculated minimum X value.
- `xmax_ref::Ref{Float64}`: Reference to store the calculated maximum X value.
- `zmin_ref::Ref{Float64}`: Reference to store the calculated minimum Z value.
- `zmax_ref::Ref{Float64}`: Reference to store the calculated maximum Z value.
"""
function bounds!(x1::Vector{Float64}, z1::Vector{Float64}, n1::Int, n2::Int,
                 xmin_ref::Ref{Float64}, xmax_ref::Ref{Float64},
                 zmin_ref::Ref{Float64}, zmax_ref::Ref{Float64})

    # Initialize min/max values using Inf/-Inf for robust comparison.
    xmax_ref[] = -Inf
    zmax_ref[] = -Inf
    xmin_ref[] = Inf
    zmin_ref[] = Inf

    # Validate input range (optional, but recommended for robustness).
    if n1 > n2
        @warn "bounds!: Starting index n1 ($n1) is greater than ending index n2 ($n2). No elements will be processed."
        return nothing
    end
    # Check if indices are within array bounds.
    if n1 < 1 || n2 > length(x1) || n2 > length(z1)
        error("bounds!: Input indices n1=$n1 or n2=$n2 are out of bounds for arrays of length $(length(x1)) or $(length(z1)).")
    end

    # Iterate through the specified range to find min and max values.
    # @inbounds skips array bounds checking for performance, assuming indices are valid.
    @inbounds for i = n1:n2
        if x1[i] < xmin_ref[]; xmin_ref[] = x1[i]; end
        if z1[i] < zmin_ref[]; zmin_ref[] = z1[i]; end
        if x1[i] > xmax_ref[]; xmax_ref[] = x1[i]; end
        if z1[i] > zmax_ref[]; zmax_ref[] = z1[i]; end
    end

    return nothing # Functions modifying arguments in-place often return nothing.
end



bounds!

In [19]:
function spl1d1(x::Vector, y::Vector, xq::Real)
    itp = CubicSplineInterpolation(x, y)
    return itp(xq)
end

function spl1d2!(x::Vector, y::Vector, xq::Real)
    itp = CubicSplineInterpolation(x, y)
    return Interpolations.gradient(itp, xq)
end

spl1d2! (generic function with 1 method)

In [ ]:


"""
    lag(ax, af, m, nl, x, iop)

This function performs Lagrange interpolation and optionally computes its derivative.

# Arguments
- `ax::AbstractVector{Float64}`: Array of x-coordinates for the interpolation points.
- `af::AbstractVector{Float64}`: Array of y-coordinates (function values) for the interpolation points.
- `m::Int`: Number of interpolation points.
- `nl::Int`: Number of points to use for the local interpolation (degree of polynomial + 1).
- `x::Float64`: The x-value at which to evaluate the interpolated function and/or its derivative.
- `iop::Int`: Operation mode:
    - `0`: Compute function value `f` only.
    - `1`: Compute derivative `df` only.
    - `2`: Compute both function value `f` and derivative `df`.

# Returns
- `f::Float64`: The interpolated function value at `x` (if `iop` is 0 or 2).
- `df::Float64`: The interpolated function derivative at `x` (if `iop` is 1 or 2).
"""
function lag!(ax::AbstractVector{Float64}, af::AbstractVector{Float64}, m::Int, nl::Int, x::Float64, iop::Int)
    f = 0.0
    df = 0.0

    # Find the index jn such that ax[jn] is closest to x
    jn = 1
    for i in 1:m
        if ax[i] >= x
            break
        end
        jn = i # jn will be the index of the largest ax[i] <= x
    end

    jn = max(jn, 1) # Ensure jn is at least 1
    if jn != m && abs(ax[jn + 1] - x) < abs(x - ax[jn])
        jn = jn + 1
    end

    # Determine the range of indices for interpolation
    jnmm = floor(Int, (nl - 0.1) / 2)
    jnpp = floor(Int, (nl + 0.1) / 2)

    nll = jn - jnmm
    nlr = jn + jnpp

    # Adjust for even nl when ax[jn] > x
    if (nl % 2 == 0) && (ax[jn] > x)
        nll -= 1
        nlr += 1
    end

    # Clamp indices to valid array bounds
    if nlr > m
        nlr = m
        nll = nlr - nl + 1
    elseif nll < 1
        nll = 1
        nlr = nl
    end

    # Compute function value f
    if iop != 1 # If iop is 0 or 2
        f = 0.0
        for i in nll:nlr
            alag = 1.0
            for j in nll:nlr
                if i == j
                    continue
                end
                alag *= (x - ax[j]) / (ax[i] - ax[j])
            end
            f += alag * af[i]
        end
        if iop == 0
            return f, df # df is 0.0 as initialized
        end
    end

    # Compute derivative df
    df = 0.0
    for i in nll:nlr
        slag = 0.0
        for id in nll:nlr
            if id == i
                continue
            end
            alag = 1.0
            for j in nll:nlr
                if j == i
                    continue
                end
                if j != id
                    alag *= (x - ax[j]) / (ax[i] - ax[j])
                else
                    alag /= (ax[i] - ax[id])
                end
            end
            slag += alag
        end
        df += slag * af[i]
    end

    return f, df
end


"""
    eqarcw(xin, zin, mw1)

This function performs arc length re-parameterization of a 2D curve. It takes an
input curve defined by `(xin, zin)` coordinates and re-samples it such that
the new points `(xout, zout)` are equally spaced in arc length.

# Arguments
- `xin::AbstractVector{Float64}`: Array of x-coordinates of the input curve.
- `zin::AbstractVector{Float64}`: Array of z-coordinates of the input curve.
- `mw1::Int`: Number of points in the input and output curves.

# Returns
- `xout::Vector{Float64}`: Array of x-coordinates of the arc-length re-parameterized curve.
- `zout::Vector{Float64}`: Array of z-coordinates of the arc-length re-parameterized curve.
- `ell::Vector{Float64}`: Array of cumulative arc lengths for the input curve.
- `thgr::Vector{Float64}`: Array of re-parameterized 'theta' values corresponding to equal arc lengths.
- `thlag::Vector{Float64}`: Array of normalized 'theta' values for the input curve (0 to 1).
"""
function eqarcw!(xin::AbstractVector{Float64}, zin::AbstractVector{Float64}, mw1::Int)
    thlag = zeros(Float64, mw1)
    ell = zeros(Float64, mw1)
    thgr = zeros(Float64, mw1)
    xout = zeros(Float64, mw1)
    zout = zeros(Float64, mw1)

    # Initialize thlag as normalized parameter from 0 to 1
    for iw in 1:mw1
        thlag[iw] = (1.0 / (mw1 - 1)) * (iw - 1)
    end

    # Calculate cumulative arc length
    ell[1] = 1.0e-8 # Small non-zero initial length
    for iw in 2:mw1
        thet = (thlag[iw] + thlag[iw - 1]) / 2.0
        
        # Get derivative of xin with respect to thlag
        _, d_xin_d_thlag = lag(thlag, xin, mw1, 3, thet, 1)
        
        # Get derivative of zin with respect to thlag
        _, d_zin_d_thlag = lag(thlag, zin, mw1, 3, thet, 1)
        
        # Calculate arc length segment
        xtzt = norm([d_xin_d_thlag, d_zin_d_thlag])
        ell[iw] = ell[iw - 1] + xtzt / (mw1 - 1)
    end

    # Re-parameterize based on equal arc lengths
    for i in 1:mw1
        elgr = (ell[mw1] / (mw1 - 1)) * (i - 1)
        
        # Interpolate thlag based on equal arc lengths
        thgr[i], _ = lag(ell, thlag, mw1, 3, elgr, 0)
    end

    # Get xout and zout using the re-parameterized thgr
    for i in 1:mw1
        ttt = thgr[i]
        
        # Interpolate xin at ttt
        xout[i], _ = lag(thlag, xin, mw1, 3, ttt, 0)
        
        # Interpolate zin at ttt
        zout[i], _ = lag(thlag, zin, mw1, 3, ttt, 0)
    end

    return xout, zout, ell, thgr, thlag
end

eqarcw

### adjustb

In [11]:
"""
    adjustb!(betin, betout_ref, a_, bw_, cw_, dw_, xmaj_, plrad_, ishape_)

Adjusts the `betin` angle based on the `ishape_` and other wall/plasma parameters.
This function takes `betout_ref` as a `Ref` so it can modify the output value in-place.

# Arguments
- `betin::Float64`: Input angle.
- `betout_ref::Ref{Float64}`: Reference to the output angle, which will be modified.
- `a_::Float64`: Wall parameter.
- `bw_::Float64`: Wall parameter (elongation or height).
- `cw_::Float64`: Wall parameter (center or offset).
- `dw_::Float64`: Wall parameter (triangularity).
- `xmaj_::Float64`: Magnetic axis X coordinate.
- `plrad_::Float64`: Plasma radius.
- `ishape_::Int`: Integer indicating the wall shape type.
"""
function adjustb!(betin::Float64, betout_ref::Ref{Float64}, a_::Float64, bw_::Float64, cw_::Float64, dw_::Float64,
                 xmaj_::Float64, plrad_::Float64, ishape_::Int)

    # Local variables (implicit real(r8) (a-h,o-z), integer (i-n) in Fortran)
    # Julia's type system is stricter, so we explicitly define types.
    local r0::Float64 = 0.0 # Initialize to avoid UndefVarError if ishape_ is not 21 or 31
    local r::Float64 = 0.0  # Initialize

    # Fortran's implicit typing for real and integer variables has been explicitly
    # typed in Julia for clarity and performance.

    if ishape_ == 31
        r0 = cw_
        r  = a_
    elseif ishape_ == 21 # Use elseif for mutually exclusive conditions
        r0 = xmaj_ + cw_ * plrad_
        r  = plrad_ * (1.0 + a_ - cw_)
    else
        # Handle cases where ishape_ is neither 21 nor 31.
        # The Fortran code doesn't explicitly define r0, r for other ishape_ values.
        # Depending on context, this might indicate an error or an unhandled case.
        # For now, they remain initialized to 0.0.
        @warn "adjustb!: Unsupported ishape_ value: $ishape_. r0 and r will remain 0.0."
    end

    bet2 = betin
    
    # Check for potential division by zero if tan(bet2) is infinite or if bw_ is zero.
    # atan(Inf) is handled correctly by Julia, but tan(bet2) could be very large
    # if bet2 is close to pi/2 + n*pi. If bw_ is zero, division by zero occurs.
    if bw_ == 0.0
        @warn "adjustb!: Division by zero detected (bw_ is 0.0). Setting betout_ref to NaN."
        betout_ref[] = NaN
        return nothing
    end

    betout_ref[] = abs(atan(tan(bet2) / bw_))

    return nothing # Fortran's `return` from a subroutine
end

adjustb!

### Test adjustb

In [ ]:

"""
    adjustb!(betin, betout_ref, a_, bw_, cw_, dw_, xmaj_, plrad_, ishape_)

Adjusts the `betin` angle based on the `ishape_` and other wall/plasma parameters.
This function takes `betout_ref` as a `Ref` so it can modify the output value in-place.

# Arguments
- `betin::Float64`: Input angle.
- `betout_ref::Ref{Float64}`: Reference to the output angle, which will be modified.
- `a_::Float64`: Wall parameter.
- `bw_::Float64`: Wall parameter (elongation or height).
- `cw_::Float64`: Wall parameter (center or offset).
- `dw_::Float64`: Wall parameter (triangularity).
- `xmaj_::Float64`: Magnetic axis X coordinate.
- `plrad_::Float64`: Plasma radius.
- `ishape_::Int`: Integer indicating the wall shape type.
"""
function adjustb!(betin::Float64, betout_ref::Ref{Float64}, a_::Float64, bw_::Float64, cw_::Float64, dw_::Float64,
                 xmaj_::Float64, plrad_::Float64, ishape_::Int)

    local r0::Float64 = 0.0
    local r::Float64 = 0.0

    if ishape_ == 31
        r0 = cw_
        r  = a_
    elseif ishape_ == 21
        r0 = xmaj_ + cw_ * plrad_
        r  = plrad_ * (1.0 + a_ - cw_)
    else
        @warn "adjustb!: Unsupported ishape_ value: $ishape_. r0 and r will remain 0.0."
    end

    bet2 = betin
    
    if bw_ == 0.0
        @warn "adjustb!: Division by zero detected (bw_ is 0.0). Setting betout_ref to NaN."
        betout_ref[] = NaN
        return nothing
    end

    betout_ref[] = abs(atan(tan(bet2) / bw_))

    return nothing
end

# --- Direct function execution and result verification ---

println("--- Starting direct execution tests for adjustb! ---")

# Common test parameters (these don't affect the core calculation of betout_ref in this function's logic)
a_val = 1.0
cw_val = 0.5
dw_val = 0.1
xmaj_val = 1.6
plrad_val = 0.5
ishape_val = 21 # Note: ishape_ does not influence the final betout calculation in this specific function

# Test Case 1: Basic operation (bw_ = 1.0)
betin_val_1 = pi / 4 # 45 degrees
bw_val_1 = 1.0
betout_ref_1 = Ref(0.0)
adjustb!(betin_val_1, betout_ref_1, a_val, bw_val_1, cw_val, dw_val, xmaj_val, plrad_val, ishape_val)
expected_1 = abs(atan(tan(betin_val_1) / bw_val_1))
println("\n--- Test Case 1: Basic Operation (betin = pi/4, bw_ = 1.0) ---")
println("Input betin: $(betin_val_1) (rad), tan(betin): $(tan(betin_val_1))")
println("Input bw_: $(bw_val_1)")
println("Calculated betout: $(betout_ref_1[])")
println("Expected betout: $(expected_1)")
if abs(betout_ref_1[] - expected_1) < 1e-9
    println("Result matches: ✅")
else
    println("Result mismatch: ❌")
end


# Test Case 2: BW_ scaling effect
betin_val_2 = pi / 4
bw_val_2 = 0.5
betout_ref_2 = Ref(0.0)
adjustb!(betin_val_2, betout_ref_2, a_val, bw_val_2, cw_val, dw_val, xmaj_val, plrad_val, ishape_val)
expected_2 = abs(atan(tan(betin_val_2) / bw_val_2))
println("\n--- Test Case 2: BW_ Scaling (betin = pi/4, bw_ = 0.5) ---")
println("Input betin: $(betin_val_2) (rad), tan(betin): $(tan(betin_val_2))")
println("Input bw_: $(bw_val_2)")
println("Calculated betout: $(betout_ref_2[])")
println("Expected betout: $(expected_2)")
if abs(betout_ref_2[] - expected_2) < 1e-9
    println("Result matches: ✅")
else
    println("Result mismatch: ❌")
end


# Test Case 3: Negative betin
betin_val_3 = -pi / 4
bw_val_3 = 1.0
betout_ref_3 = Ref(0.0)
adjustb!(betin_val_3, betout_ref_3, a_val, bw_val_3, cw_val, dw_val, xmaj_val, plrad_val, ishape_val)
expected_3 = abs(atan(tan(betin_val_3) / bw_val_3))
println("\n--- Test Case 3: Negative betin (betin = -pi/4, bw_ = 1.0) ---")
println("Input betin: $(betin_val_3) (rad), tan(betin): $(tan(betin_val_3))")
println("Input bw_: $(bw_val_3)")
println("Calculated betout: $(betout_ref_3[])")
println("Expected betout: $(expected_3)")
if abs(betout_ref_3[] - expected_3) < 1e-9
    println("Result matches: ✅")
else
    println("Result mismatch: ❌")
end

# Test Case 4: bw_ = 0.0 (Error handling)
betin_val_4 = pi / 4
bw_val_4 = 0.0
betout_ref_4 = Ref(0.0)
println("\n--- Test Case 4: bw_ = 0.0 (Expected Warning and NaN) ---")
# You'll see the @warn message printed to the console.
# isnan() function checks if the result is NaN.
adjustb!(betin_val_4, betout_ref_4, a_val, bw_val_4, cw_val, dw_val, xmaj_val, plrad_val, ishape_val)
println("Input betin: $(betin_val_4) (rad)")
println("Input bw_: $(bw_val_4)")
println("Calculated betout: $(betout_ref_4[])")
if isnan(betout_ref_4[])
    println("Result is NaN (as expected): ✅")
else
    println("Result is not NaN (Error): ❌")
end

# Test Case 5: Unsupported ishape_
betin_val_5 = pi / 6
bw_val_5 = 1.0
betout_ref_5 = Ref(0.0)
ishape_val_5 = 99 # Unsupported value
println("\n--- Test Case 5: Unsupported ishape_ (Expected Warning) ---")
adjustb!(betin_val_5, betout_ref_5, a_val, bw_val_5, cw_val, dw_val, xmaj_val, plrad_val, ishape_val_5)
expected_5 = abs(atan(tan(betin_val_5) / bw_val_5))
println("Input ishape_: $(ishape_val_5)")
println("Calculated betout: $(betout_ref_5[])")
println("Expected betout (calculation formula is the same): $(expected_5)")
if abs(betout_ref_5[] - expected_5) < 1e-9
    println("Result matches (only warning issued): ✅")
else
    println("Result mismatch: ❌")
end

println("\n--- Direct execution tests for adjustb! completed ---")

--- Starting direct execution tests for adjustb! ---

--- Test Case 1: Basic Operation (betin = pi/4, bw_ = 1.0) ---
Input betin: 0.7853981633974483 (rad), tan(betin): 0.9999999999999999
Input bw_: 1.0
Calculated betout: 0.7853981633974483
Expected betout: 0.7853981633974483
Result matches: ✅

--- Test Case 2: BW_ Scaling (betin = pi/4, bw_ = 0.5) ---
Input betin: 0.7853981633974483 (rad), tan(betin): 0.9999999999999999
Input bw_: 0.5
Calculated betout: 1.1071487177940904
Expected betout: 1.1071487177940904
Result matches: ✅

--- Test Case 3: Negative betin (betin = -pi/4, bw_ = 1.0) ---
Input betin: -0.7853981633974483 (rad), tan(betin): -0.9999999999999999
Input bw_: 1.0
Calculated betout: 0.7853981633974483
Expected betout: 0.7853981633974483
Result matches: ✅

--- Test Case 4: bw_ = 0.0 (Expected Warning and NaN) ---
Input betin: 0.7853981633974483 (rad)
Input bw_: 0.0
Calculated betout: NaN
Result is NaN (as expected): ✅

--- Test Case 5: Unsupported ishape_ (Expected Warning) ---

┌ Warning: adjustb!: Division by zero detected (bw_ is 0.0). Setting betout_ref to NaN.
└ @ Main In[12]:39
┌ Warning: adjustb!: Unsupported ishape_ value: 99. r0 and r will remain 0.0.
└ @ Main In[12]:33


### d3dvesl, d3dwall

In [ ]:
"""
    d3dvesl!(r0, z0, a0, e0, ar, az, nval, zst, r, z, npts, ier_ref)

Defines the shape of the DIII-D vacuum vessel based on Fourier series coefficients.

# Arguments
- `r0::Float64`: Reference major radius.
- `z0::Float64`: Reference vertical position.
- `a0::Float64`: Minor radius scaling factor.
- `e0::Float64`: Elongation scaling factor.
- `ar::Vector{Float64}`: Fourier coefficients for radial component.
- `az::Vector{Float64}`: Fourier coefficients for vertical component.
- `nval::Int`: Number of Fourier coefficients.
- `zst::Float64`: Starting vertical position for calculation (used in an unused branch).
- `r::Vector{Float64}`: Output vector for R coordinates of the wall (modified in-place).
- `z::Vector{Float64}`: Output vector for Z coordinates of the wall (modified in-place).
- `npts::Int`: Number of points to generate for the wall.
- `ier_ref::Ref{Int}`: Error flag (0 for success, 1 for error) passed by reference.
"""
function d3dvesl!(r0::Float64, z0::Float64, a0::Float64, e0::Float64,
                 ar::Vector{Float64}, az::Vector{Float64}, nval::Int, zst::Float64,
                 r::Vector{Float64}, z::Vector{Float64}, npts::Int, ier_ref::Ref{Int})

    pii = acos(-1.0) # Using Julia's built-in pi constant from Pye
    ier_ref[] = 0 # Initialize error flag

    local arcst::Float64

    if abs(z0 - zst) <= 1.0e-6
        # This branch is usually taken, as zstart is set to 0.0 in d3dwall
        arcst = 0.0
    else
        local isgn::Int
        local zpcmp::Float64
        local arci::Float64
        local arcf::Float64
        local dfi::Float64
        local arca::Float64
        local zza::Float64
        local arcb::Float64
        local zzb::Float64
        local ackb::Float64
        local arcc::Float64
        local zzc::Float64
        local ackc::Float64
        local dzp::Float64
        local dzm::Float64
        local dzt::Float64
        local dcf1::Float64
        local dcf2::Float64
        local zdf::Float64

        if z0 < zst
            isgn = +1
        end
        if z0 > zst
            isgn = -1
        end
        zpcmp = (zst - z0) / (a0 * e0)
        arci = 0.0
        arcf = 0.5 * pii
        dfi = (arcf - arci) / npts
        arca = arci
        zza = 0.0
        for j = 2:npts
            arcb = arca + isgn * dfi
            zzb = 0.0
            for k = 1:nval
                ackb = k * arcb
                zzb = zzb + az[k] * sin(ackb)
            end # 5 continue
            if (zza - zpcmp) * (zzb - zpcmp) <= 0.0
                arcc = arcb + isgn * dfi
                zzc = 0.0
                for k = 1:nval
                    ackc = k * arcc
                    zzc = zzc + az[k] * sin(ackc)
                end # 10 continue
                @goto label_25
            else
                arca = arcb
                zza = zzb
            end
        end # 20 continue
        ier_ref[] = 1
        return
        @label label_25 # 25 continue
        dzp = zzc - zzb
        dzm = zzb - zza
        dzt = zzc - zza
        dcf1 = dfi * (dzm / dzp + dzp / dzm) / dzt
        dcf2 = dfi * (1.0 / dzp - 1.0 / dzm) / dzt
        zdf = zpcmp - zzb
        arcst = arcb + dcf1 * zdf + dcf2 * zdf * zdf
    end

    arc0 = arcst
    arc1 = arcst + 2.0 * pii
    darc = (arc1 - arc0) / npts

    for j = 1:npts
        arc = arc0 + (j - 1.0) * darc
        sumr = 0.0
        sumz = 0.0
        for k = 1:nval
            arck = k * arc
            sumr = sumr + ar[k] * cos(arck)
            sumz = sumz + az[k] * sin(arck)
        end # 50 continue
        rpval = r0 + a0 * sumr
        zpval = z0 - e0 * a0 * sumz
        r[j] = rpval
        z[j] = zpval
    end # 100 continue

    return nothing
end

In [ ]:
using Printf # For @printf

"""
    d3dwall!(xwall, ywall, mthh, iomod, iotty1, rext_in, zlim_val_in)

Defines the shape of the DIII-D wall by calling `d3dvesl`.

# Arguments
- `xwall::Vector{Float64}`: Output vector for X coordinates of the DIII-D wall (modified in-place).
- `ywall::Vector{Float64}`: Output vector for Y (Z in Cartesian) coordinates of the DIII-D wall (modified in-place).
- `mthh::Int`: Number of points to generate for the wall.
- `iomod::IO`: Output stream for general messages.
- `iotty1::IO`: Output stream for terminal messages.
- `rext_in::Float64`: External scaling factor for the wall.
- `zlim_val_in::Float64`: Global z-limit value (used in `d3dvesl` implicitly).
"""
function d3dwall!(xwall::Vector{Float64}, ywall::Vector{Float64}, mthh::Int,
                 iomod::IO, iotty1::IO, rext_in::Float64, zlim_val_in::Float64)

    ncdf = 26
    rwi = [
        1.0, 0.05526794, -0.1738114, 0.01850757, 0.03714965, -0.02882647,
        -0.002357329, 0.009548103, -0.01214923, -0.001853416, 0.006837493,
        -0.001711245, 0.002270762, 0.003689963, -0.003959393, -0.001098017,
        0.003745465, -0.0002157904, -0.0003977743, -0.0002725623,
        -0.001005857, -0.000004579016, 0.002396789, -0.0007057043,
        0.001158347, 0.0003552319
    ]
    zwi = [
        1.0, -0.03236632, -0.1629422, 0.06013983, 0.01167756, -0.02579542,
        0.01626464, -0.002085857, -0.009098639, 0.01022163, -0.004388253,
        -0.009367258, 0.008308497, 0.004765150, -0.004611675, -0.001121423,
        -0.0002501100, 0.4282634e-03, 0.2669702e-02, -0.1073800e-02, # Corrected: -0.1073800d-02 from Fortran
        -0.2191338e-02, 0.1328267e-02, 0.5050959e-03, -0.5758863e-03,
        0.9348883e-03, 0.7094351e-03
    ]

    rwall0 = 1.6400000
    zwall0 = 0.0000000
    awall0 = 0.8839410
    ewall0 = 1.4037020

    nwcoef = ncdf

    rwll = rwall0
    zwll = zwall0
    awll = awall0 * rext_in # Use rext_in here
    ewll = ewall0

    # In the Fortran code, `zstart = zlim` and then `zlim = 0.0`, followed by `zstart = 0.0`.
    # This effectively makes `zstart` 0.0 for the call to `d3dvesl`.
    zstart = 0.0 # This overwrites zlim_val_in for the purpose of d3dvesl.

    nwalp = mthh

    ier_ref = Ref(0) # Julia's way to pass an integer by reference for modification
    d3dvesl(rwll, zwll, awll, ewll, rwi, zwi, nwcoef, zstart, xwall, ywall, nwalp, ier_ref)
    ier = ier_ref[] # Get the value from the reference

    @printf(iomod, "ier in d3dwall = %3d\n", ier)
    @printf(iotty1, "ier in d3dwall = %3d\n", ier)

    xwall[mthh+1] = xwall[1]
    ywall[mthh+1] = ywall[1]
end

d3dwall!

### Test d3dwall, d3dvesl

In [ ]:

# 1. Define input parameters
# mthh is the number of points for the wall, also equivalent to npts for d3dvesl!
mthh_val = 200 # A reasonable number of points for a smooth curve
rext_val = 1.0 # External scaling factor, usually 1.0 for standard DIII-D wall
zlim_val = 0.0 # This value is overwritten to 0.0 inside d3dwall! for zstart

# 2. Initialize output vectors
# xwall and ywall need to be mthh_val + 1 long because d3dwall! adds the first point at the end
xwall_output = Vector{Float64}(undef, mthh_val + 1)
ywall_output = Vector{Float64}(undef, mthh_val + 1)

# 3. Set up IO streams
# `devnull` suppresses output. Use `stdout` to see messages in the console.
iomod_stream = devnull # You can change this to stdout if you want to see "ier in d3dwall" for iomod
iotty1_stream = stdout # Prints "ier in d3dwall" to your terminal

# 4. Call the d3dwall! function
println("Calling d3dwall! with mthh_val = $mthh_val and rext_val = $rext_val...")
d3dwall!(xwall_output, ywall_output, mthh_val, iomod_stream, iotty1_stream, rext_val, zlim_val)
println("d3dwall! call completed. Check output below:")

# 5. Verify the results
println("\n--- Verification ---")

# Check if the output vectors are populated
@printf "Is xwall_output empty? %s\n" isempty(xwall_output) ? "Yes" : "No"
@printf "Is ywall_output empty? %s\n" isempty(ywall_output) ? "Yes" : "No"

# Check the lengths
@printf "Length of xwall_output: %d (Expected: %d)\n" length(xwall_output) (mthh_val + 1)
@printf "Length of ywall_output: %d (Expected: %d)\n" length(ywall_output) (mthh_val + 1)

# Check if the first and last points match (as d3dwall! explicitly sets them)
# Use ≈ for approximate equality with floating-point numbers
@printf "xwall_output[1] (%f) vs xwall_output[end] (%f): Match? %s\n" xwall_output[1] xwall_output[end] (xwall_output[1] ≈ xwall_output[end]) ? "Yes" : "No"
@printf "ywall_output[1] (%f) vs ywall_output[end] (%f): Match? %s\n" ywall_output[1] ywall_output[end] (ywall_output[1] ≈ ywall_output[end]) ? "Yes" : "No"

# Print a few sample points to visually inspect the shape (optional, for debugging)
println("\n--- Sample Points (R, Z) ---")
for i in 1:min(5, mthh_val)
    @printf "Point %3d: R = %8.4f, Z = %8.4f\n" i xwall_output[i] ywall_output[i]
end
println("...")
for i in max(1, mthh_val-4):mthh_val+1
    @printf "Point %3d: R = %8.4f, Z = %8.4f\n" i xwall_output[i] ywall_output[i]
end

using Plots
plot(xwall_output, ywall_output, seriestype=:scatter, legend=false, aspect_ratio=:equal,
     xlabel="R (m)", ylabel="Z (m)", title="DIII-D Vacuum Vessel Shape")
savefig("d3d_vessel_shape.png") # Saves the plot to a file
println("\nPlot saved to d3d_vessel_shape.png")

Calling d3dwall! with mthh_val = 200 and rext_val = 1.0...
ier in d3dwall =   0
d3dwall! call completed. Check output below:

--- Verification ---
Is xwall_output empty? No
Is ywall_output empty? No
Length of xwall_output: 201 (Expected: 201)
Length of ywall_output: 201 (Expected: 201)
xwall_output[1] (2.446648) vs xwall_output[end] (2.446648): Match? Yes
ywall_output[1] (0.000000) vs ywall_output[end] (0.000000): Match? Yes

--- Sample Points (R, Z) ---
Point   1: R =   2.4466, Z =   0.0000
Point   2: R =   2.4459, Z =  -0.0268
Point   3: R =   2.4440, Z =  -0.0528
Point   4: R =   2.4418, Z =  -0.0778
Point   5: R =   2.4405, Z =  -0.1022
...
Point 196: R =   2.4405, Z =   0.1269
Point 197: R =   2.4405, Z =   0.1022


Point 198: R =   2.4418, Z =   0.0778
Point 199: R =   2.4440, Z =   0.0528
Point 200: R =   2.4459, Z =   0.0268
Point 201: R =   2.4466, Z =   0.0000

Plot saved to d3d_vessel_shape.png


### wwall

In [7]:
# Global variables from vglobal_mod (Illustrative - you would define these based on your vglobal_mod contents)
# These should ideally be defined once at the top level of your script or in a configuration file.
# I'm using `Ref` for single mutable global values where Fortran might pass by reference
# or for values that are modified within the subroutine.

# Constants and initial values (replace with actual values from your setup)
const nths = 100 # Example value
const pye = π
global aw = 0.0 # Will be saved and restored
global bw = 0.0 # Will be saved and restored
global a = 0.0
global b = 0.0
global cw = 0.0
global dw = 0.0
global tw = 0.0
global abulg = 0.0
global bbulg = 0.0
global tbulg = 0.0
global ishape = 0
global farwal = false
global leqarcw = 0

# Placeholder for arrays that would be in vglobal_mod
global xinf = zeros(Float64, nths) # Assuming size nths based on context
global zinf = zeros(Float64, nths) # Assuming size nths based on context
global xma = 0.0
global zma = 0.0

# Define IO streams (replace with your actual stream definitions)
const outmod = stdout # or open("outmod.log", "w")
const iotty = stdout  # or open("/dev/tty", "w")
const outpest = stderr # or open("outpest.err", "w")

# External Fortran subroutines that are expected to be available or have Julia equivalents
# If these are actual Fortran compiled subroutines, you'd use `ccall`.
# Assuming you have Julia equivalents for these already as per your prompt.
# function adjustb!(...) defined in helper.
# function d3dwall!(...)
# function d3dvesl!(...)
# (lag!, eqarcw!, bounds!, errmes! are provided as helpers by the user)


"""
    wwall!(nqnqnq::Int, xwal1::Vector{Float64}, zwal1::Vector{Float64})

This subroutine calculates the coordinates of the wall (xwal1, zwal1) based on
various geometric shapes and plasma parameters. It modifies `xwal1` and `zwal1` in place.

# Arguments
- `nqnqnq::Int`: An integer parameter (its specific use isn't clear from the snippet,
                  but it's passed as an argument).
- `xwal1::Vector{Float64}`: Array to store the calculated x-coordinates of the wall.
                            Modified in-place.
- `zwal1::Vector{Float64}`: Array to store the calculated z-coordinates of the wall.
                            Modified in-place.

# Global Variables Used/Modified (from vglobal_mod in Fortran)
- `a`, `aw`, `b`, `bw`, `cw`, `dw`, `tw`, `abulg`, `bbulg`, `tbulg`, `ishape`
- `farwal`, `leqarcw`
- `xinf`, `zinf`, `xma`, `zma`
- `outmod`, `iotty`, `outpest`
"""
function wwall!(nqnqnq::Int, xwal1::Vector{Float64}, zwal1::Vector{Float64})
    # Fortran's implicit real(r8) (a-h,o-z) and implicit integer (i-n) are
    # handled by Julia's type system and explicit variable declarations.
    # r8 corresponds to Float64.

    # Local variable declarations
    # Variables that were POINTER in Fortran need careful handling.
    # In Julia, if they are temporary arrays that get allocated/deallocated,
    # just create them as local arrays.
    local npots0::Int
    local npots::Int
    local lfix::Bool
    local insect::Bool
    local csmin::Float64
    local thetatmp::Vector{Float64}
    local xwaltmp::Vector{Float64}
    local zwaltmp::Vector{Float64}
    local xpptmp::Vector{Float64}
    local ww1tmp::Vector{Float64}
    local ww2tmp::Vector{Float64}
    local ww3tmp::Vector{Float64}
    local tabtmp::Vector{Float64}
    local rioptmp::Vector{Int}

    # Dimensions based on Fortran code:
    # dimension iop(2),xpp(nths),zpp(nths),ww1(nths),ww2(nths),
    #     ww3(nths),thet(nths),tabx(3),tabz(3)
    # The Fortran code passes nths as an assumed-size array argument (nths),
    # but also uses it as a fixed size for some local arrays.
    # We'll use `nths` (the global constant) for array sizing.
    iop = Vector{Int}(undef, 2) # Undef because data iplt/0/ means iop is not explicitly initialized here.
    xpp = zeros(Float64, nths)
    zpp = zeros(Float64, nths)
    ww1 = zeros(Float64, nths)
    ww2 = zeros(Float64, nths)
    ww3 = zeros(Float64, nths)
    thet = zeros(Float64, nths)
    tabx = zeros(Float64, 3)
    tabz = zeros(Float64, 3)

    # Fortran DATA statement
    iplt = 0

    # Variables for common calculations
    local awsave::Float64
    local bwsave::Float64
    local isph::Int
    local xmnp::Ref{Float64} = Ref(0.0) # Use Ref for single value output from `bounds!`
    local xmxp::Ref{Float64} = Ref(0.0)
    local zmnp::Ref{Float64} = Ref(0.0)
    local zmxp::Ref{Float64} = Ref(0.0)
    local xmin::Float64
    local xmax::Float64
    local zmin::Float64
    local zmax::Float64
    local plrad::Float64
    local xmaj::Float64
    local zmid::Float64
    local zrad::Float64
    local scale::Float64
    local delta1::Float64
    local mthalf::Int
    local inside::Int = 0 # Initialize `inside` as it's incremented

    # Constants calculated based on nths
    mth = nths # Assuming nths is equivalent to mth for indexing purposes in this context
    mth1 = nths # mth1 = mth + 1 in Fortran
    mth2 = nths # mth2 = mth + 2 in Fortran
    dth = 2.0 * pye / (mth2 - 1) # (2.0*pi / (mth+1))

    # --- Computations ---
    xpp[1] = 1.0
    zpp[1] = 1.0
    ww3[1] = 1.0
    tabx[1] = 1.0
    tabz[1] = 1.0

    awsave = aw
    bwsave = bw
    insect = false
    isph = 0

    # --- Shape Logic ---
    if a < -100.0
        isph = 1
        ishape = -10
        # Assuming xinf and zinf are accessible global arrays
        bounds!(xinf, zinf, 1, mth, xmnp, xmxp, zmnp, zmxp)
        xmin = xmnp[]
        xmax = xmxp[]
        zmin = zmnp[]
        zmax = zmxp[]
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        zmid = 0.5 * (zmax + zmin)
        hrad = xmax + aw * (xmax - xmaj)
        vrad = zmax + bw * (zmax - zmid)

        for i in 1:mth1
            xi = xinf[i] - xmaj
            zeta = zinf[i] - zmid
            bbb = (xi * vrad)^2 + (zeta * hrad)^2
            ccc = -xmaj * vrad * xi + hrad * sqrt(bbb - (zeta * xmaj)^2)
            xwal1[i] = xmaj + xi * vrad * ccc / bbb
            zwal1[i] = zmid + zeta * vrad * ccc / bbb
        end
        @goto cleanup # Direct jump to cleanup section
    end

    if a > -10.0
        lfix = true
    else
        lfix = false # Initialize lfix for all paths
    end

    if farwal
        return # early exit because there is no wall
    end

    xshift = a

    mthalf = floor(Int, mth2 / 2) # Fortran integer division

    zmin = 3.4e38
    zmax = -3.4e38
    xmin = 3.4e38
    xmax = -3.4e38
    for i in 1:mth
        if xmax < xinf[i]; xmax = xinf[i]; end
        if xmin > xinf[i]; xmin = xinf[i]; end
        if zmax < zinf[i]; zmax = zinf[i]; end
        if zmin > zinf[i]; zmin = zinf[i]; end
    end
    plrad = 0.5 * (xmax - xmin)
    xmaj = 0.5 * (xmax + xmin)
    zmid = 0.5 * (zmax + zmin)
    zrad = 0.5 * (zmax - zmin)
    scale = (zmax - zmin)
    if ((xmax - xmin) / 2.0) > scale
        scale = (xmax - xmin) / 2.0
    end
    scale = 1.0
    aw = aw * scale
    bw = bw * scale
    delta1 = dw * (xinf[1] - xma)

    # ishape=2 Elliptical shell
    if ishape == 2
        zh = sqrt(abs(zrad^2 - plrad^2)) # h metric
        zah = a / zh
        zph = plrad / zh
        zmup = 0.5 * log((zrad + plrad) / (zrad - plrad)) # mu-plas
        zmuw = log(zah + sqrt(zah^2 + 1)) # mu-wall
        zxmup = exp(zmup)
        zxmuw = exp(zmuw)
        zbwal = zh * cosh(zmuw) # Major radius of wall
        bw = zbwal / a          # Elongation of wall
        for i in 1:mth2
            the = (i - 1) * dth
            xwal1[i] = xmaj + a * cos(the)
            zwal1[i] = -bw * a * sin(the)
        end
        @info outmod println("Confocal Ellipse:\nmup, expmup = %13.5e %13.5e\nmuw, expmuw = %13.5e %13.5e\n", zmup, zxmup, zmuw, zxmuw)
        @info iotty println("Confocal Ellipse:\nmup, expmup = %13.5e %13.5e\nmuw, expmuw = %13.5e %13.5e\n", zmup, zxmup, zmuw, zxmuw)
    end

    # ishape=3
    if ishape == 3
        for i in 1:mth2
            rr = (xinf[i] - xma)^2 + (zinf[i] - zma)^2
            ro = sqrt(rr)
            the = atan((zinf[i] - zma), (xinf[i] - xma)) # Use atan2 for correct quadrant
            thex = abs(the)
            lsgn = 1
            if xma > xinf[i]
                the = the + pye
            end
            if i <= mthalf
                if xma > xinf[i]
                    thex = pye - thex
                end
                thet[i] = abs(thex)
            end
            if !lfix
                ro = ro + delta1
                xwal1[i] = xma + lsgn * ro * cos(the)
                zwal1[i] = zma + lsgn * ro * sin(the)
            else
                xshift = (xmax + xmin) / 2.0
                xshift = a
                the = (i - 1) * dth
                xwal1[i] = xshift + aw * cos(the + dw * sin(the))
                zwal1[i] = zma - bw * sin(the)
            end
            if i > mthalf
                continue
            end
            if zwal1[i] < zmin
                continue
            end
            j = i
            insect = false
            jsmall = j
            jlarge = j
            ics = 1
            if xma >= xinf[i]
                ics = -1
            end
            while true
                if zinf[j] >= zwal1[i]
                    jsmall = j
                else
                    break
                end
                if j >= mthalf
                    continue # Fortran's cycle
                end
                if j < 1
                    continue # Fortran's cycle
                end
                j = j + ics
            end
            jlarge = j
            if abs(xinf[jsmall] - xma) >= abs(xwal1[i] - xma)
                insect = true
            end
            if abs(xinf[jlarge] - xma) >= abs(xwal1[i] - xma)
                insect = true
            end
            if !insect
                continue
            end
            inside += 1
        end
    end

    # ishape=4 Modified dee-shaped wall independent of plasma geometry
    if ishape == 4
        wcentr = cw
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            sn2th = sin(2.0 * the)
            xwal1[i] = cw + a * cos(the + dw * sin(the))
            zwal1[i] = -bw * a * sin(the + tw * sn2th) - aw * sn2th
        end
    end

    # ishape=5 Dee-shaped wall scaled to plasma
    if ishape == 5
        wcentr = xmaj + cw * plrad
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            sn2th = sin(2.0 * the)
            xwal1[i] = xmaj + cw * plrad + plrad * (1.0 + a - cw) * cos(the + dw * sin(the))
            zwal1[i] = -bw * plrad * (1.0 + a - cw) * sin(the + tw * sn2th) - aw * plrad * sn2th
        end
    end

    # ishape=6 Conforming shell
    if ishape == 6
        wcentr = xmaj
        # Fortran's minval(xinf(2:mth1))
        # Note: If xinf is guaranteed to have at least 2 elements, this is safe.
        csmin = min(0.1, 1e-1 * minimum(view(xinf, 2:mth1)))
        for i in 2:mth1
            alph = atan(xinf[i+1] - xinf[i-1], zinf[i-1] - zinf[i+1]) # Fortran's ATAN2
            xwal1[i] = xinf[i] + a * plrad * cos(alph)
            zwal1[i] = zinf[i] + a * plrad * sin(alph)
            # if the wall crosses the R=0 axis, force a thin center stack
            if xwal1[i] <= csmin
                xwal1[i] = csmin
            end
        end
        xwal1[1] = xwal1[mth1]
        zwal1[1] = zwal1[mth1]
        xwal1[mth2] = xwal1[2]
        zwal1[mth2] = zwal1[2]
    end

    # ishape=7 Enclosing bean-shaped wall
    if ishape == 7
        cwr = cw * pye / 180.0
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            rho = aw * (1.0 + bw * cos(the))
            the2 = cwr * sin(the)
            xofsw = xmax + a * plrad - aw * (1.0 + bw)
            xwal1[i] = xofsw + rho * cos(the2)
            zwal1[i] = -b * rho * sin(the2)
        end
    end

    # ishape=8 Wall of DIII-D
    if ishape == 8
        # Ensure d3dwall! takes xwal1, zwal1, mth, outmod, iotty as arguments
        d3dwall!(xwal1, zwal1, mth, outmod, iotty)
    end

    # ishape=11 Dee-shaped conductor
    if ishape == 11
        for i in 1:mth2
            the = (i - 1) * dth
            plrad = 0.5 * (xmax - xmin)
            xwal1[i] = xmax + plrad * (a + aw - aw * cos(the + dw * sin(the)))
            zwal1[i] = -plrad * bw * sin(the)
        end
    end

    # ishape=12 Solid bean-shaped conductor on right
    if ishape == 12
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        brad = b * pye / 180.0
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            rho = a0 - aw * plrad * cos(the)
            the2 = brad * sin(the)
            xwal1[i] = xmaj + cw * plrad + rho * cos(the2)
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=13 Solid bean-shaped conductor on left
    if ishape == 13
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        brad = b * pye / 180.0
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            rho = a0 + aw * plrad * cos(the)
            the2 = brad * sin(the)
            xwal1[i] = xmaj + cw * plrad - rho * cos(the2)
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=21 Shell scaled to plasma. Gap on the inner side.
    if ishape == 21
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        a0b = (a0 + plrad * aw) * bw
        brad0 = b * pye / 180.0
        brad = brad0
        blgrad0 = bbulg * pye / 180.0
        wcentr = xmaj + cw * plrad

        # Create Ref for adjustb! output
        blgrado_ref = Ref(0.0)
        blgradi_ref = Ref(0.0)

        # Call adjustb!
        adjustb!(blgrad0, blgrado_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgrado = blgrado_ref[]

        dthb = (2.0 * aw * plrad / a0b) * (1.0 - sin(blgrado)) / cos(blgrado)
        blgrad0 = blgrad0 - dthb
        
        adjustb!(blgrad0, blgradi_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgradi = blgradi_ref[]

        for i in 1:mth2
            the0 = (i - 1) * dth
            thbulg = (the0 > 0.5 * pye && the0 < 1.5 * pye) ? blgrado : blgradi
            cost2b = cos(2 * thbulg)
            the = the0
            cost = cos(the)
            ferm = +1.0 - 2.0 / (exp(cost / tw) + 1.0)
            rho = a0 - aw * plrad * ferm
            the2 = brad * sin(the)
            cost2 = cos(2.0 * the2)
            fermb = 1.0 / (exp((cost2b - cost2) / tbulg) + 1.0)
            bulge = abulg * plrad * fermb
            xwal1[i] = xmaj + cw * plrad + rho * cos(the2 + dw * sin(the2)) + bulge
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=24 Shell scaled to plasma. Gap on the outer side.
    if ishape == 24
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        brad = b * pye / 180.0
        wcentr = xmaj + cw * plrad
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            cost = cos(the)
            ferm = +1.0 - 2.0 / (exp(cost / tw) + 1.0)
            rho = a0 + aw * plrad * ferm
            the2 = brad * sin(the)
            xwal1[i] = xmaj + cw * plrad - rho * cos(the2 - dw * sin(the2))
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=31 Shell independent of plasma. Gap on the inner side.
    if ishape == 31
        a0 = a + aw
        a0b = (a0 + aw) * bw
        brad0 = b * pye / 180.0
        brad = brad0
        blgrad0 = bbulg * pye / 180.0
        wcentr = cw

        blgrado_ref = Ref(0.0)
        blgradi_ref = Ref(0.0)

        adjustb!(blgrad0, blgrado_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgrado = blgrado_ref[]

        dthb = (2.0 * aw / a0b) * (1.0 - sin(blgrado)) / cos(blgrado)
        blgrad0 = blgrad0 - dthb
        
        adjustb!(blgrad0, blgradi_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgradi = blgradi_ref[]

        for i in 1:mth2
            the0 = (i - 1) * dth
            thbulg = (the0 > 0.5 * pye && the0 < 1.5 * pye) ? blgrado : blgradi
            cost2b = cos(2.0 * thbulg)
            the = the0
            cost = cos(the)
            ferm = +1.0 - 2.0 / (exp(cost / tw) + 1.0)
            rho = a0 - aw * ferm
            the2 = brad * sin(the)
            cost2 = cos(2.0 * the2)
            fermb = 1.0 / (exp((cost2b - cost2) / tbulg) + 1.0)
            bulge = abulg * fermb
            xwal1[i] = cw + rho * cos(the2 + dw * sin(the2)) + bulge
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=34 Shell independent of plasma. Gap on the outer side.
    if ishape == 34
        a0 = a + aw
        brad = b * pye / 180.0
        wcentr = cw
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            cost = cos(the)
            ferm = +1.0 - 2.0 / (exp(cost / tw) + 1.0)
            rho = a0 + aw * ferm
            the2 = brad * sin(the)
            xwal1[i] = cw - rho * cos(the2 - dw * sin(the2))
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=41 Arbitrary wall generated by spline data
    if ishape == 41
        # In Julia, opening files is straightforward.
        # We need to define `spl1d1` and `spl1d2` functions if they aren't provided.
        # Assuming they are available (or you will provide them).

        # Temporary arrays for pointers
        # These are local variables that will be collected by GC when they go out of scope.
        # npots = npots0 + 5 - 1 is a bit odd in Fortran, implies npots0 is the actual size used.
        # Let's assume npots0 is the number of data points read from the file.
        open("wall_geo.in", "r") do io
            npots0 = parse(Int, readline(io))
            wcentr = parse(Float64, readline(io))
            readline(io) # Skip the next line

            # Allocate arrays based on npots0, as per Fortran's dynamic allocation
            thetatmp = zeros(Float64, npots0)
            xwaltmp = zeros(Float64, npots0)
            zwaltmp = zeros(Float64, npots0)
            rioptmp = zeros(Int, 2)
            xpptmp = zeros(Float64, npots0)
            ww1tmp = zeros(Float64, npots0)
            ww2tmp = zeros(Float64, npots0)
            ww3tmp = zeros(Float64, npots0)
            tabtmp = zeros(Float64, 3)

            for i in 1:npots0
                line = split(readline(io))
                thetatmp[i] = parse(Float64, line[1])
                xwaltmp[i] = parse(Float64, line[2]) - wcentr
                zwaltmp[i] = parse(Float64, line[3])
            end
        end # `do io` block automatically closes the file

        rioptmp[1] = 4
        rioptmp[2] = 4
        # Assuming spl1d1 and spl1d2 are defined elsewhere, similar to lag and eqarcw
        # This will need actual implementation of spl1d1 and spl1d2
        # Example call: spl1d1!(npots0, thetatmp, xwaltmp, xpptmp, rioptmp, 1, ww1tmp, ww2tmp, ww3tmp)
        # spl1d1 needs to be a function that takes these arguments.
        # You'll need to define `spl1d1!` and `spl1d2!` if you don't have them yet.
        @warn "spl1d1! and spl1d2! are called but not defined in this snippet. Please ensure they are available."

        # Placeholder calls for spl1d1 and spl1d2
        # For a full conversion, you'd need the Fortran source for spl1d1 and spl1d2
        # For now, let's assume they exist as `spl1d1!` and `spl1d2!` (mutating versions)
        # call spl1d1(npots0,thetatmp,xwaltmp,xpptmp,rioptmp,1,ww1tmp,ww2tmp,ww3tmp)
        # call spl1d1(npots0,thetatmp,zwaltmp,xpptmp,rioptmp,1,ww1tmp,ww2tmp,ww3tmp)

        for i in 1:mth1
            the0 = (i - 1) * dth
            # call spl1d2(npots0,thetatmp,xwaltmp,xpptmp,1,the0,tabtmp)
            # Assuming spl1d2! fills tabtmp
            # If spl1d2 returns the value, modify the call appropriately
            # xwal1[i] = spl1d2_value_for_x * a + wcentr
            xwal1[i] = 0.0 # Placeholder
        end

        rioptmp[1] = 4
        rioptmp[2] = 4
        # call spl1d1(npots0,thetatmp,zwaltmp,xpptmp,rioptmp,1,ww1tmp,ww2tmp,ww3tmp)

        for i in 1:mth1
            the0 = (i - 1) * dth
            # call spl1d2(npots0,thetatmp,zwaltmp,xpptmp,1,the0,tabtmp)
            # zwal1[i] = spl1d2_value_for_z * a
            zwal1[i] = 0.0 # Placeholder
        end

        xwal1[1] = xwal1[mth1]
        zwal1[1] = zwal1[mth1]
        xwal1[mth2] = xwal1[2]
        zwal1[mth2] = zwal1[2]

        # Deallocation is automatic in Julia (Garbage Collection)
        # The variables `thetatmp`, `xwaltmp`, etc., will be garbage collected
        # when they go out of scope or are no longer referenced.
    end

    # ishape=42 Arbitrary wall generated by position data
    if ishape == 42
        open("wall_geo.in", "r") do io
            npots0 = parse(Int, readline(io))
            wcentr = parse(Float64, readline(io))
            readline(io) # Skip the next line

            if npots0 != mth + 2
                @error "ERROR: Number of points in wall_geo.in must be equal to mth+2."
                error("Wall geometry error") # Stop execution
            end

            # thetatmp is read but ignored in Fortran; here it's still needed to parse the line
            thetatmp_dummy = zeros(Float64, npots0)
            for i in 1:npots0
                line = split(readline(io))
                thetatmp_dummy[i] = parse(Float64, line[1]) # Read but not used
                xwal1[i] = parse(Float64, line[2])
                zwal1[i] = parse(Float64, line[3])
            end
        end

        if xwal1[mth1] != xwal1[1] || zwal1[mth1] != zwal1[1]
            @error "ERROR: First point in wall_geo.in must be equal to 2nd last point."
            error("Wall geometry error")
        end
        if xwal1[mth2] != xwal1[2] || zwal1[mth2] != zwal1[2]
            @error "ERROR: Last point in wall_geo.in must be equal to 2nd point."
            error("Wall geometry error")
        end
    end

    xmx = xma + xshift

    # --- This section is never called ---
    # The Fortran code has `if( .false. ) then ... endif`, so this block is dead code.
    # We will omit it in Julia.

    # --- Cleanup ---
    @label cleanup # Target for the `goto` from ishape=-10
    if leqarcw == 1
        # Call eqarcw! and update xwal1, zwal1
        xpp_out, zpp_out, ww1_out, ww2_out, ww3_out = eqarcw!(xwal1, zwal1, mth1)
        for i in 1:mth1
            xwal1[i] = xpp_out[i]
            zwal1[i] = zpp_out[i]
        end
    end

    # Assuming `savewall` is defined or you handle writing the wall shape to disk.
    # If not defined, you'll need to implement it.
    @warn "savewall! is called but not defined in this snippet. Please ensure it is available."
    # savewall!(wcentr, xwal1, zwal1) # Write wall shape to disk

    if iplt <= 0
        xmx = xmaj
        zma = 0.0
        iplt = 1
    end


    aw = awsave # restore value of aw
    bw = bwsave # restore value of bw

    return nothing # Fortran subroutines don't return a value
end

wwall!

In [ ]:
using LinearAlgebra: norm # For sqrt(x^2 + y^2)
using Printf # For @sprintf used in logging

# --- 1. Global variables from vglobal_mod (Fortran의 USE vglobal_mod 역할) ---
# 이 변수들은 wwall! 함수가 접근할 수 있도록 전역 범위에 정의되어야 합니다.
# 실제 Fortran 코드의 값에 맞게 초기화해야 합니다.

global nths = 100 # 플라즈마/벽 점의 개수 (예시)
global aw = 0.1 # 벽 오프셋 파라미터 (예시)
global bw = 1.2 # 벽 연신율 파라미터 (예시)
global a = 1.0 # 주 반지름 또는 벽 스케일링 인자 (예시)
global b = 0.0 # 삼각도 파라미터 (예시)
global cw = 0.0 # 벽 중심 이동 (예시)
global dw = 0.0 # 벽 삼각도 (예시)
global tw = 0.1 # 벽 스퀘어니스 (예시)
global abulg = 0.0 # 돌출부 크기 (예시)
global bbulg = 0.0 # 돌출부 각도 (예시)
global tbulg = 0.1 # 돌출부 전환 폭 (예시)
global ishape = 0 # 벽 형태를 제어하는 변수 (테스트 시 변경됨)
global farwal = false # 벽이 멀리 있는지 여부 (false면 리턴하지 않음)
global leqarcw = 1 # 아크 길이 재매개변수화 수행 여부 (1: 수행, 0: 안함)

# 플라즈마 입력 형태 배열 (nths 점) - 테스트 시 채워집니다.
global xinf = zeros(Float64, nths)
global zinf = zeros(Float64, nths)
global xma = 0.0 # 플라즈마 중심 X (예시)
global zma = 0.0 # 플라즈마 중심 Z (예시)

# IO 스트림 (Fortran의 UNIT 역할) - 테스트를 위해 stdout으로 설정
const outmod = stdout
const iotty = stdout
const outpest = stdout

# --- 2. 헬퍼 함수들 (이전 턴에서 제공되었거나, 여기에서 더미로 구현) ---

"""
    errmes(io::IO, subprog_name::String)

`errmes` 서브루틴의 더미 구현입니다.
지정된 IO 스트림에 오류 메시지를 출력합니다.
"""
function errmes(io::IO, subprog_name::String)
    @error "$subprog_name: 오류가 발생했습니다."
end


"""
    bounds!(x1::Vector{Float64}, z1::Vector{Float64}, n1::Int, n2::Int,
            xmin_ref::Ref{Float64}, xmax_ref::Ref{Float64},
            zmin_ref::Ref{Float64}, zmax_ref::Ref{Float64})

두 입력 벡터의 지정된 범위 내에서 최소/최대 X, Z 좌표를 계산합니다.
이 함수는 제공된 `Ref` 인자를 제자리에서 수정합니다.
"""
function bounds!(x1::Vector{Float64}, z1::Vector{Float64}, n1::Int, n2::Int,
                 xmin_ref::Ref{Float64}, xmax_ref::Ref{Float64},
                 zmin_ref::Ref{Float64}, zmax_ref::Ref{Float64})

    xmax_ref[] = -Inf
    zmax_ref[] = -Inf
    xmin_ref[] = Inf
    zmin_ref[] = Inf

    if n1 > n2
        @warn "bounds!: 시작 인덱스 n1 ($n1)이 끝 인덱스 n2 ($n2)보다 큽니다. 아무 요소도 처리되지 않습니다."
        return nothing
    end
    if n1 < 1 || n2 > length(x1) || n2 > length(z1)
        error("bounds!: 입력 인덱스 n1=$n1 또는 n2=$n2가 배열 길이 $(length(x1)) 또는 $(length(z1))의 범위를 벗어났습니다.")
    end

    @inbounds for i = n1:n2
        if x1[i] < xmin_ref[]; xmin_ref[] = x1[i]; end
        if z1[i] < zmin_ref[]; zmin_ref[] = z1[i]; end
        if x1[i] > xmax_ref[]; xmax_ref[] = x1[i]; end
        if z1[i] > zmax_ref[]; zmax_ref[] = z1[i]; end
    end
    return nothing
end

"""
    lag!(ax, af, m, nl, x, iop)

라그랑주 보간법을 수행하고 선택적으로 미분을 계산합니다.
(f, df) 튜플을 반환합니다.
"""
function lag!(ax::AbstractVector{Float64}, af::AbstractVector{Float64}, m::Int, nl::Int, x::Float64, iop::Int)
    f = 0.0
    df = 0.0

    jn = 1
    for i in 1:m
        if ax[i] >= x
            break
        end
        jn = i
    end

    jn = max(jn, 1)
    if jn != m && abs(ax[jn + 1] - x) < abs(x - ax[jn])
        jn = jn + 1
    end

    jnmm = floor(Int, (nl - 0.1) / 2)
    jnpp = floor(Int, (nl + 0.1) / 2)

    nll = jn - jnmm
    nlr = jn + jnpp

    if (nl % 2 == 0) && (ax[jn] > x)
        nll -= 1
        nlr += 1
    end

    if nlr > m
        nlr = m
        nll = nlr - nl + 1
    elseif nll < 1
        nll = 1
        nlr = nl
    end

    if iop != 1
        f = 0.0
        for i in nll:nlr
            alag = 1.0
            for j in nll:nlr
                if i == j
                    continue
                end
                alag *= (x - ax[j]) / (ax[i] - ax[j])
            end
            f += alag * af[i]
        end
        if iop == 0
            return f, df
        end
    end

    df = 0.0
    for i in nll:nlr
        slag = 0.0
        for id in nll:nlr
            if id == i
                continue
            end
            alag = 1.0
            for j in nll:nlr
                if j == i
                    continue
                end
                if j != id
                    alag *= (x - ax[j]) / (ax[i] - ax[j])
                else
                    alag /= (ax[i] - ax[id])
                end
            end
            slag += alag
        end
        df += slag * af[i]
    end

    return f, df
end

"""
    eqarcw!(xin, zin, mw1)

2D 곡선의 아크 길이 재매개변수화를 수행합니다.
(xout, zout, ell, thgr, thlag) 튜플을 반환합니다.
"""
function eqarcw!(xin::AbstractVector{Float64}, zin::AbstractVector{Float64}, mw1::Int)
    thlag = zeros(Float64, mw1)
    ell = zeros(Float64, mw1)
    thgr = zeros(Float64, mw1)
    xout = zeros(Float64, mw1)
    zout = zeros(Float64, mw1)

    for iw in 1:mw1
        thlag[iw] = (1.0 / (mw1 - 1)) * (iw - 1)
    end

    ell[1] = 1.0e-8
    for iw in 2:mw1
        thet = (thlag[iw] + thlag[iw - 1]) / 2.0
        
        _, d_xin_d_thlag = lag!(thlag, xin, mw1, 3, thet, 1)
        _, d_zin_d_thlag = lag!(thlag, zin, mw1, 3, thet, 1)
        
        xtzt = norm([d_xin_d_thlag, d_zin_d_thlag])
        ell[iw] = ell[iw - 1] + xtzt / (mw1 - 1)
    end

    for i in 1:mw1
        elgr = (ell[mw1] / (mw1 - 1)) * (i - 1)
        thgr[i], _ = lag!(ell, thlag, mw1, 3, elgr, 0)
    end

    for i in 1:mw1
        ttt = thgr[i]
        xout[i], _ = lag!(thlag, xin, mw1, 3, ttt, 0)
        zout[i], _ = lag!(thlag, zin, mw1, 3, ttt, 0)
    end

    return xout, zout, ell, thgr, thlag
end


"""
    adjustb!(betin::Float64, betout_ref::Ref{Float64}, a_::Float64, bw_::Float64, cw_::Float64, dw_::Float64,
             xmaj_::Float64, plrad_::Float64, ishape_::Int)

`adjustb!`의 더미 구현입니다. 실제 Fortran 함수는 여기에 복잡한 계산을 수행할 수 있습니다.
"""
function adjustb!(betin::Float64, betout_ref::Ref{Float64}, a_::Float64, bw_::Float64, cw_::Float64, dw_::Float64,
                 xmaj_::Float64, plrad_::Float64, ishape_::Int)
    betout_ref[] = betin # 단순 더미: 입력값을 그대로 반환
    @info "adjustb! 호출됨: betin=$betin, a_=$a_, bw_=$bw_, ishape_=$ishape_. betout_ref[]=$(betout_ref[])."
    return nothing
end

"""
    d3dwall!(xwal1::Vector{Float64}, zwal1::Vector{Float64}, mth::Int, outmod::IO, iotty::IO)

`d3dwall!`의 더미 구현입니다.
"""
function d3dwall!(xwal1::Vector{Float64}, zwal1::Vector{Float64}, mth::Int, outmod::IO, iotty::IO)
    @info iotty "d3dwall! 호출됨. 이것은 더미 함수입니다. xwal1과 zwal1은 여기서 수정되지 않습니다."
    return nothing
end

"""
    savewall!(wcentr::Float64, xwal1::Vector{Float64}, zwal1::Vector{Float64})

`savewall!`의 더미 구현입니다.
"""
function savewall!(wcentr::Float64, xwal1::Vector{Float64}, zwal1::Vector{Float64})
    @info "savewall! 호출됨. 벽 형태 (처음 5점): x=[$(xwal1[1:min(5, end)])], z=[$(zwal1[1:min(5, end)])]. wcentr=$wcentr"
    return nothing
end

"""
    spl1d1!(npots0::Int, thetatmp::Vector{Float64}, xwaltmp::Vector{Float64}, xpptmp::Vector{Float64},
            rioptmp::Vector{Int}, iop_val::Int, ww1tmp::Vector{Float64}, ww2tmp::Vector{Float64}, ww3tmp::Vector{Float64})

`spl1d1!`의 더미 구현입니다.
"""
function spl1d1!(npots0::Int, thetatmp::Vector{Float64}, xwaltmp::Vector{Float64}, xpptmp::Vector{Float64},
                 rioptmp::Vector{Int}, iop_val::Int, ww1tmp::Vector{Float64}, ww2tmp::Vector{Float64}, ww3tmp::Vector{Float64})
    @info "spl1d1! 더미 호출됨. 입력 데이터가 복사되거나 간단하게 변형됩니다."
    # 실제 스플라인 계산 대신, 간단한 더미 작업을 수행합니다.
    # 예: xpptmp에 xwaltmp의 복사본을 넣거나, ww1tmp, ww2tmp, ww3tmp에 간단한 값을 채웁니다.
    xpptmp .= xwaltmp # 더미 복사
    ww1tmp .= 1.0
    ww2tmp .= 2.0
    ww3tmp .= 3.0
    return nothing
end

"""
    spl1d2!(npots0::Int, thetatmp::Vector{Float64}, xwaltmp::Vector{Float64}, xpptmp::Vector{Float64},
            iop_val::Int, the0::Float64, tabtmp::Vector{Float64})

`spl1d2!`의 더미 구현입니다.
"""
function spl1d2!(npots0::Int, thetatmp::Vector{Float64}, xwaltmp::Vector{Float64}, xpptmp::Vector{Float64},
                 iop_val::Int, the0::Float64, tabtmp::Vector{Float64})
    @info "spl1d2! 더미 호출됨. tabtmp에 간단한 값을 채웁니다. the0=$the0"
    # 실제 스플라인 보간 계산 대신, 간단한 더미 작업을 수행합니다.
    tabtmp[1] = xwaltmp[mod1(round(Int, the0 * (npots0 - 1) / (2*pye)) + 1, npots0)] # 대략적인 보간
    tabtmp[2] = 0.1 # 더미 미분
    tabtmp[3] = 0.01 # 더미 2차 미분
    return nothing
end

# --- wwall! 함수 (이전 턴에서 Julia로 번역된 코드) ---
# 이 함수는 이전 응답에서 제공된 Julia 코드와 동일합니다.

function wwall!(nqnqnq::Int, xwal1::Vector{Float64}, zwal1::Vector{Float64})
    local npots0::Int
    local npots::Int
    local lfix::Bool
    local insect::Bool
    local csmin::Float64
    local thetatmp::Vector{Float64}
    local xwaltmp::Vector{Float64}
    local zwaltmp::Vector{Float64}
    local xpptmp::Vector{Float64}
    local ww1tmp::Vector{Float64}
    local ww2tmp::Vector{Float64}
    local ww3tmp::Vector{Float64}
    local tabtmp::Vector{Float64}
    local rioptmp::Vector{Int}

    iop = Vector{Int}(undef, 2)
    xpp = zeros(Float64, nths)
    zpp = zeros(Float64, nths)
    ww1 = zeros(Float64, nths)
    ww2 = zeros(Float64, nths)
    ww3 = zeros(Float64, nths)
    thet = zeros(Float64, nths)
    tabx = zeros(Float64, 3)
    tabz = zeros(Float64, 3)

    iplt = 0

    local awsave::Float64
    local bwsave::Float64
    local isph::Int
    local xmnp::Ref{Float64} = Ref(0.0)
    local xmxp::Ref{Float64} = Ref(0.0)
    local zmnp::Ref{Float64} = Ref(0.0)
    local zmxp::Ref{Float64} = Ref(0.0)
    local xmin::Float64
    local xmax::Float64
    local zmin::Float64
    local zmax::Float64
    local plrad::Float64
    local xmaj::Float64
    local zmid::Float64
    local zrad::Float64
    local scale::Float64
    local delta1::Float64
    local mthalf::Int
    local inside::Int = 0

    mth = nths
    mth1 = nths
    mth2 = nths
    dth = 2.0 * pye / (mth2 - 1)

    xpp[1] = 1.0
    zpp[1] = 1.0
    ww3[1] = 1.0
    tabx[1] = 1.0
    tabz[1] = 1.0

    awsave = aw
    bwsave = bw
    insect = false
    isph = 0

    if a < -100.0
        isph = 1
        ishape = -10
        bounds!(xinf, zinf, 1, mth, xmnp, xmxp, zmnp, zmxp)
        xmin = xmnp[]
        xmax = xmxp[]
        zmin = zmnp[]
        zmax = zmxp[]
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        zmid = 0.5 * (zmax + zmin)
        hrad = xmax + aw * (xmax - xmaj)
        vrad = zmax + bw * (zmax - zmid)

        for i in 1:mth1
            xi = xinf[i] - xmaj
            zeta = zinf[i] - zmid
            bbb = (xi * vrad)^2 + (zeta * hrad)^2
            ccc = -xmaj * vrad * xi + hrad * sqrt(bbb - (zeta * xmaj)^2)
            # bbb가 0이 될 가능성 처리 (예: 분모가 0이 되는 경우)
            if abs(bbb) < eps(Float64) # 아주 작은 값으로 0 체크
                xwal1[i] = NaN
                zwal1[i] = NaN
                @warn "ishape=-10: Division by zero avoided, result set to NaN for point $i. Check input values."
            else
                xwal1[i] = xmaj + xi * vrad * ccc / bbb
                zwal1[i] = zmid + zeta * vrad * ccc / bbb
            end
        end
        @goto cleanup
    end

    if a > -10.0
        lfix = true
    else
        lfix = false
    end

    if farwal
        return
    end

    xshift = a

    mthalf = floor(Int, mth2 / 2)

    zmin = 3.4e38
    zmax = -3.4e38
    xmin = 3.4e38
    xmax = -3.4e38
    for i in 1:mth
        if xmax < xinf[i]; xmax = xinf[i]; end
        if xmin > xinf[i]; xmin = xinf[i]; end
        if zmax < zinf[i]; zmax = zinf[i]; end
        if zmin > zinf[i]; zmin = zinf[i]; end
    end
    plrad = 0.5 * (xmax - xmin)
    xmaj = 0.5 * (xmax + xmin)
    zmid = 0.5 * (zmax + zmin) # Fortran 코드의 zmid 정의와 일치

    zrad = 0.5 * (zmax - zmin)
    scale = (zmax - zmin)
    if ((xmax - xmin) / 2.0) > scale
        scale = (xmax - xmin) / 2.0
    end
    scale = 1.0 # Fortran 코드에 명시된 1.0으로 재설정
    aw = aw * scale
    bw = bw * scale
    delta1 = dw * (xinf[1] - xma)

    if ishape == 2
        zh = sqrt(abs(zrad^2 - plrad^2))
        # zh가 0이 되는 경우를 피함
        if abs(zh) < eps(Float64)
            @warn "ishape=2: zh is effectively zero. Ellipse calculations might be problematic."
            zah = Inf # 또는 다른 적절한 값
        else
            zah = a / zh
        end
        zph = plrad / zh # zh가 0이면 여기서도 문제
        
        # log 인자가 음수나 0이 되는 경우를 피함
        log_arg1 = (zrad + plrad) / (zrad - plrad)
        if log_arg1 <= 0
            @warn "ishape=2: log argument 1 <= 0. zmup might be NaN/Inf."
            zmup = NaN
        else
            zmup = 0.5 * log(log_arg1)
        end
        
        log_arg2 = zah + sqrt(zah^2 + 1)
        if log_arg2 <= 0
            @warn "ishape=2: log argument 2 <= 0. zmuw might be NaN/Inf."
            zmuw = NaN
        else
            zmuw = log(log_arg2)
        end
        
        zxmup = exp(zmup)
        zxmuw = exp(zmuw)
        zbwal = zh * cosh(zmuw)
        
        # a가 0이 되는 경우를 피함
        if abs(a) < eps(Float64)
            @warn "ishape=2: Division by zero avoided for bw calculation. Setting bw=NaN."
            bw = NaN
        else
            bw = zbwal / a
        end

        for i in 1:mth2
            the = (i - 1) * dth
            xwal1[i] = xmaj + a * cos(the)
            zwal1[i] = -bw * a * sin(the)
        end
        @info outmod @sprintf("Confocal Ellipse:\nmup, expmup = %13.5e %13.5e\nmuw, expmuw = %13.5e %13.5e\n", zmup, zxmup, zmuw, zxmuw)
        @info iotty @sprintf("Confocal Ellipse:\nmup, expmup = %13.5e %13.5e\nmuw, expmuw = %13.5e %13.5e\n", zmup, zxmup, zmuw, zxmuw)
    end

    if ishape == 3
        for i in 1:mth2
            rr = (xinf[i] - xma)^2 + (zinf[i] - zma)^2
            ro = sqrt(rr)
            the = atan(zinf[i] - zma, xinf[i] - xma) # atan2 사용
            thex = abs(the)
            lsgn = 1
            if xma > xinf[i]
                the = the + pye
            end
            if i <= mthalf
                if xma > xinf[i]
                    thex = pye - thex
                end
                thet[i] = abs(thex)
            end
            if !lfix
                ro = ro + delta1
                xwal1[i] = xma + lsgn * ro * cos(the)
                zwal1[i] = zma + lsgn * ro * sin(the)
            else
                xshift = (xmax + xmin) / 2.0
                xshift = a
                the = (i - 1) * dth
                xwal1[i] = xshift + aw * cos(the + dw * sin(the))
                zwal1[i] = zma - bw * sin(the)
            end
            if i > mthalf
                continue
            end
            if zwal1[i] < zmin
                continue
            end
            j = i
            insect = false
            jsmall = j
            jlarge = j
            ics = 1
            if xma >= xinf[i]
                ics = -1
            end
            while true
                if zinf[j] >= zwal1[i]
                    jsmall = j
                else
                    break
                end
                # Fortran의 cycle은 다음 루프 반복으로 이동합니다.
                # 여기서는 if 조건이 만족하면 현재 while 루프의 다음 반복으로.
                if j >= mthalf
                    j = j + ics # Fortran의 j = j + ics; cycle
                    continue
                end
                if j < 1
                    j = j + ics # Fortran의 j = j + ics; cycle
                    continue
                end
                j = j + ics
            end
            jlarge = j
            if abs(xinf[jsmall] - xma) >= abs(xwal1[i] - xma)
                insect = true
            end
            if abs(xinf[jlarge] - xma) >= abs(xwal1[i] - xma)
                insect = true
            end
            if !insect
                continue
            end
            inside += 1
        end
    end

    if ishape == 4
        wcentr = cw
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            sn2th = sin(2.0 * the)
            xwal1[i] = cw + a * cos(the + dw * sin(the))
            zwal1[i] = -bw * a * sin(the + tw * sn2th) - aw * sn2th
        end
    end

    if ishape == 5
        wcentr = xmaj + cw * plrad
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            sn2th = sin(2.0 * the)
            xwal1[i] = xmaj + cw * plrad + plrad * (1.0 + a - cw) * cos(the + dw * sin(the))
            zwal1[i] = -bw * plrad * (1.0 + a - cw) * sin(the + tw * sn2th) - aw * plrad * sn2th
        end
    end

    if ishape == 6
        wcentr = xmaj
        csmin = min(0.1, 1e-1 * minimum(view(xinf, 2:mth1)))
        for i in 2:mth1
            alph = atan(xinf[i+1] - xinf[i-1], zinf[i-1] - zinf[i+1])
            xwal1[i] = xinf[i] + a * plrad * cos(alph)
            zwal1[i] = zinf[i] + a * plrad * sin(alph)
            if xwal1[i] <= csmin
                xwal1[i] = csmin
            end
        end
        xwal1[1] = xwal1[mth1]
        zwal1[1] = zwal1[mth1]
        xwal1[mth2] = xwal1[2]
        zwal1[mth2] = zwal1[2]
    end

    if ishape == 7
        cwr = cw * pye / 180.0
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            rho = aw * (1.0 + bw * cos(the))
            the2 = cwr * sin(the)
            xofsw = xmax + a * plrad - aw * (1.0 + bw)
            xwal1[i] = xofsw + rho * cos(the2)
            zwal1[i] = -b * rho * sin(the2)
        end
    end

    if ishape == 8
        d3dwall!(xwal1, zwal1, mth, outmod, iotty)
    end

    if ishape == 11
        for i in 1:mth2
            the = (i - 1) * dth
            plrad = 0.5 * (xmax - xmin)
            xwal1[i] = xmax + plrad * (a + aw - aw * cos(the + dw * sin(the)))
            zwal1[i] = -plrad * bw * sin(the)
        end
    end

    if ishape == 12
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        brad = b * pye / 180.0
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            rho = a0 - aw * plrad * cos(the)
            the2 = brad * sin(the)
            xwal1[i] = xmaj + cw * plrad + rho * cos(the2)
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    if ishape == 13
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        brad = b * pye / 180.0
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            rho = a0 + aw * plrad * cos(the)
            the2 = brad * sin(the)
            xwal1[i] = xmaj + cw * plrad - rho * cos(the2)
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    if ishape == 21
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        # a0b가 0이 되는 경우를 피함
        if abs(bw) < eps(Float64)
            @warn "ishape=21: bw is effectively zero. a0b will be zero, causing division issues."
            a0b = NaN # 또는 다른 적절한 값
        else
            a0b = (a0 + plrad * aw) * bw
        end
        
        brad0 = b * pye / 180.0
        brad = brad0
        blgrad0 = bbulg * pye / 180.0
        wcentr = xmaj + cw * plrad

        blgrado_ref = Ref(0.0)
        blgradi_ref = Ref(0.0)

        adjustb!(blgrad0, blgrado_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgrado = blgrado_ref[]

        # cos(blgrado)가 0이 되는 경우를 피함
        cos_blgrado = cos(blgrado)
        if abs(cos_blgrado) < eps(Float64)
            @warn "ishape=21: cos(blgrado) is effectively zero. dthb will be Inf/NaN."
            dthb = NaN
        else
            dthb = (2.0 * aw * plrad / a0b) * (1.0 - sin(blgrado)) / cos_blgrado
        end
        
        blgrad0 = blgrad0 - dthb
        
        adjustb!(blgrad0, blgradi_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgradi = blgradi_ref[]

        for i in 1:mth2
            the0 = (i - 1) * dth
            thbulg = (the0 > 0.5 * pye && the0 < 1.5 * pye) ? blgrado : blgradi
            cost2b = cos(2 * thbulg)
            the = the0
            cost = cos(the)
            # exp(cost/tw) + 1.0 이 0이 되는 경우를 피함
            exp_term_tw = exp(cost / tw) + 1.0
            if abs(exp_term_tw) < eps(Float64)
                @warn "ishape=21: exp(cost/tw)+1.0 is effectively zero. ferm will be problematic."
                ferm = NaN
            else
                ferm = +1.0 - 2.0 / exp_term_tw
            end
            
            rho = a0 - aw * plrad * ferm
            the2 = brad * sin(the)
            cost2 = cos(2.0 * the2)
            # exp((cost2b - cost2)/tbulg) + 1.0 이 0이 되는 경우를 피함
            exp_term_tbulg = exp((cost2b - cost2) / tbulg) + 1.0
            if abs(exp_term_tbulg) < eps(Float64)
                @warn "ishape=21: exp((cost2b-cost2)/tbulg)+1.0 is effectively zero. fermb will be problematic."
                fermb = NaN
            else
                fermb = 1.0 / exp_term_tbulg
            end
            
            bulge = abulg * plrad * fermb
            xwal1[i] = xmaj + cw * plrad + rho * cos(the2 + dw * sin(the2)) + bulge
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    if ishape == 24
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        brad = b * pye / 180.0
        wcentr = xmaj + cw * plrad
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            cost = cos(the)
            exp_term_tw = exp(cost / tw) + 1.0
            if abs(exp_term_tw) < eps(Float64)
                @warn "ishape=24: exp(cost/tw)+1.0 is effectively zero. ferm will be problematic."
                ferm = NaN
            else
                ferm = +1.0 - 2.0 / exp_term_tw
            end
            rho = a0 + aw * plrad * ferm
            the2 = brad * sin(the)
            xwal1[i] = xmaj + cw * plrad - rho * cos(the2 - dw * sin(the2))
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    if ishape == 31
        a0 = a + aw
        # a0b가 0이 되는 경우를 피함
        if abs(bw) < eps(Float64)
            @warn "ishape=31: bw is effectively zero. a0b will be zero, causing division issues."
            a0b = NaN
        else
            a0b = (a0 + aw) * bw
        end
        
        brad0 = b * pye / 180.0
        brad = brad0
        blgrad0 = bbulg * pye / 180.0
        wcentr = cw

        blgrado_ref = Ref(0.0)
        blgradi_ref = Ref(0.0)

        adjustb!(blgrad0, blgrado_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgrado = blgrado_ref[]

        cos_blgrado = cos(blgrado)
        if abs(cos_blgrado) < eps(Float64)
            @warn "ishape=31: cos(blgrado) is effectively zero. dthb will be Inf/NaN."
            dthb = NaN
        else
            dthb = (2.0 * aw / a0b) * (1.0 - sin(blgrado)) / cos_blgrado
        end
        
        blgrad0 = blgrad0 - dthb
        
        adjustb!(blgrad0, blgradi_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgradi = blgradi_ref[]

        for i in 1:mth2
            the0 = (i - 1) * dth
            thbulg = (the0 > 0.5 * pye && the0 < 1.5 * pye) ? blgrado : blgradi
            cost2b = cos(2.0 * thbulg)
            the = the0
            cost = cos(the)
            exp_term_tw = exp(cost / tw) + 1.0
            if abs(exp_term_tw) < eps(Float64)
                @warn "ishape=31: exp(cost/tw)+1.0 is effectively zero. ferm will be problematic."
                ferm = NaN
            else
                ferm = +1.0 - 2.0 / exp_term_tw
            end
            rho = a0 - aw * ferm
            the2 = brad * sin(the)
            cost2 = cos(2.0 * the2)
            exp_term_tbulg = exp((cost2b - cost2) / tbulg) + 1.0
            if abs(exp_term_tbulg) < eps(Float64)
                @warn "ishape=31: exp((cost2b-cost2)/tbulg)+1.0 is effectively zero. fermb will be problematic."
                fermb = NaN
            else
                fermb = 1.0 / exp_term_tbulg
            end
            bulge = abulg * fermb
            xwal1[i] = cw + rho * cos(the2 + dw * sin(the2)) + bulge
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    if ishape == 34
        a0 = a + aw
        brad = b * pye / 180.0
        wcentr = cw
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            cost = cos(the)
            exp_term_tw = exp(cost / tw) + 1.0
            if abs(exp_term_tw) < eps(Float64)
                @warn "ishape=34: exp(cost/tw)+1.0 is effectively zero. ferm will be problematic."
                ferm = NaN
            else
                ferm = +1.0 - 2.0 / exp_term_tw
            end
            rho = a0 + aw * ferm
            the2 = brad * sin(the)
            xwal1[i] = cw - rho * cos(the2 - dw * sin(the2))
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    if ishape == 41
        @warn "ishape=41 분기가 호출되었지만 spl1d1! 및 spl1d2!가 완전히 구현되지 않았습니다. 의미 있는 결과를 생성하지 못할 수 있습니다."
        open("wall_geo.in", "r") do io
            npots0 = parse(Int, readline(io))
            wcentr = parse(Float64, readline(io))
            readline(io)

            thetatmp = zeros(Float64, npots0)
            xwaltmp = zeros(Float64, npots0)
            zwaltmp = zeros(Float64, npots0)
            rioptmp = zeros(Int, 2)
            xpptmp = zeros(Float64, npots0)
            ww1tmp = zeros(Float64, npots0)
            ww2tmp = zeros(Float64, npots0)
            ww3tmp = zeros(Float64, npots0)
            tabtmp = zeros(Float64, 3)

            for i in 1:npots0
                line = split(readline(io))
                thetatmp[i] = parse(Float64, line[1])
                xwaltmp[i] = parse(Float64, line[2]) - wcentr
                zwaltmp[i] = parse(Float64, line[3])
            end
        end

        rioptmp[1] = 4
        rioptmp[2] = 4
        spl1d1!(npots0, thetatmp, xwaltmp, xpptmp, rioptmp, 1, ww1tmp, ww2tmp, ww3tmp)
        for i in 1:mth1
            the0 = (i - 1) * dth
            spl1d2!(npots0, thetatmp, xwaltmp, xpptmp, 1, the0, tabtmp)
            xwal1[i] = tabtmp[1] * a + wcentr
        end
        rioptmp[1] = 4
        rioptmp[2] = 4
        spl1d1!(npots0, thetatmp, zwaltmp, xpptmp, rioptmp, 1, ww1tmp, ww2tmp, ww3tmp)
        for i in 1:mth1
            the0 = (i - 1) * dth
            spl1d2!(npots0, thetatmp, zwaltmp, xpptmp, 1, the0, tabtmp)
            zwal1[i] = tabtmp[1] * a
        end
        xwal1[1] = xwal1[mth1]
        zwal1[1] = zwal1[mth1]
        xwal1[mth2] = xwal1[2]
        zwal1[mth2] = zwal1[2]
    end

    if ishape == 42
        @warn "ishape=42 분기가 호출되었습니다. 'wall_geo.in' 파일이 필요합니다."
        open("wall_geo.in", "r") do io
            npots0 = parse(Int, readline(io))
            wcentr = parse(Float64, readline(io))
            readline(io)

            if npots0 != mth + 2
                @error "ERROR: wall_geo.in의 점 개수는 mth+2와 같아야 합니다."
                error("Wall geometry error")
            end

            thetatmp_dummy = zeros(Float64, npots0)
            for i in 1:npots0
                line = split(readline(io))
                thetatmp_dummy[i] = parse(Float64, line[1])
                xwal1[i] = parse(Float64, line[2])
                zwal1[i] = parse(Float64, line[3])
            end
        end

        if xwal1[mth1] != xwal1[1] || zwal1[mth1] != zwal1[1]
            @error "ERROR: wall_geo.in의 첫 번째 점은 뒤에서 두 번째 점과 같아야 합니다."
            error("Wall geometry error")
        end
        if xwal1[mth2] != xwal1[2] || zwal1[mth2] != zwal1[2]
            @error "ERROR: wall_geo.in의 마지막 점은 두 번째 점과 같아야 합니다."
            error("Wall geometry error")
        end
    end

    xmx = xma + xshift

    @label cleanup
    if leqarcw == 1
        xpp_out, zpp_out, ww1_out, ww2_out, ww3_out = eqarcw!(xwal1, zwal1, mth1)
        for i in 1:mth1
            xwal1[i] = xpp_out[i]
            zwal1[i] = zpp_out[i]
        end
    end

    savewall!(wcentr, xwal1, zwal1)

    if iplt <= 0
        xmx = xmaj
        zma = 0.0
        iplt = 1
    end

    if insect
        @info iotty @sprintf("플라즈마에 최소 %3d개의 벽 점이 있습니다.\n", inside)
        @info outpest @sprintf("플라즈마에 최소 %3d개의 벽 점이 있습니다.\n", inside)
        @info outmod @sprintf("플라즈마에 최소 %3d개의 벽 점이 있습니다.\n", inside)
        errmes(outpest, "vacdat")
    end

    global aw = awsave
    global bw = bwsave

    return nothing
end


# --- 3. 테스트 스크립트 실행 ---
function run_wwall_test()
    println("--- wwall! 테스트 시작 ---")

    # 3.1. 테스트 케이스를 위한 전역 파라미터 초기화
    global nths = 100
    global aw = 0.2
    global bw = 1.5
    global a = 1.5 # 벽 스케일링
    global b = 0.1
    global cw = 0.0
    global dw = 0.1
    global tw = 0.05
    global abulg = 0.0
    global bbulg = 0.0
    global tbulg = 0.1
    global farwal = false
    global leqarcw = 1 # 아크 길이 재매개변수화 활성화

    # 3.2. xinf, zinf (더미 플라즈마 형태: 원) 채우기
    # ishape=2의 경우, 플라즈마 경계가 xinf, zinf에서 파생됩니다.
    # X=1.0, Z=0.0에 반지름 0.3인 플라즈마를 만듭니다.
    plasma_center_x = 1.0
    plasma_center_z = 0.0
    plasma_radius = 0.3

    for i in 1:nths
        angle = (i - 1) * 2 * pye / (nths - 1)
        xinf[i] = plasma_center_x + plasma_radius * cos(angle)
        zinf[i] = plasma_center_z + plasma_radius * sin(angle)
    end
    
    # xma, zma는 전역 변수이며, Fortran 코드에서 플라즈마 중심을 나타내는 데 사용될 수 있습니다.
    global xma = plasma_center_x
    global zma = plasma_center_z

    # 3.3. xwal1, zwal1 초기화 (출력 배열)
    xwal1 = zeros(Float64, nths + 2) # Fortran의 mth2 크기
    zwal1 = zeros(Float64, nths + 2)

    println("\n--- 테스트 케이스 1: ishape = 2 (Elliptical shell) ---")
    global ishape = 2 # ishape=2 (타원형 쉘) 테스트
    println("초기 플라즈마 파라미터 (xinf[1], zinf[1]): ($(xinf[1]), $(zinf[1]))")
    println("전역 `a` (벽 스케일링): $a, `aw` (벽 오프셋): $aw, `bw` (벽 연신율): $bw")
    println("`ishape` = $ishape 테스트 중입니다.")

    # 3.4. wwall! 호출
    nqnqnq_dummy = 0 # 이 인자의 목적은 명확하지 않으므로 더미 값 사용
    wwall!(nqnqnq_dummy, xwal1, zwal1)


    println("\n--- wwall! 테스트 결과 1 ---")
    println("첫 5개 X-벽 점: $(xwal1[1:min(5, end)])")
    println("첫 5개 Z-벽 점: $(zwal1[1:min(5, end)])")
    println("마지막 5개 X-벽 점: $(xwal1[max(1, end-4):end])")
    println("마지막 5개 Z-벽 점: $(zwal1[max(1, end-4):end])")
    println("최종 전역 `aw` 값: $aw")
    println("최종 전역 `bw` 값: $bw")

    if all(iszero, xwal1) && ishape != 0
        println("경고: xwal1이 여전히 모두 0입니다. 벽 형태가 올바르게 계산되지 않았거나 해당 경로가 실행되지 않았을 수 있습니다.")
    else
        println("xwal1이 채워진 것으로 보입니다 (모두 0은 아님).")
    end


    # 3.5. 다른 ishape 테스트 (예: ishape=6 Conforming shell)
    println("\n--- 테스트 케이스 2: ishape = 6 (Conforming shell) ---")
    global ishape = 6
    global a = 0.1 # 컨포밍 쉘의 작은 간격
    fill!(xwal1, 0.0) # xwal1, zwal1 초기화
    fill!(zwal1, 0.0)
    wwall!(nqnqnq_dummy, xwal1, zwal1)
    println("첫 5개 X-벽 점 (ishape 6): $(xwal1[1:min(5, end)])")
    println("첫 5개 Z-벽 점 (ishape 6): $(zwal1[1:min(5, end)])")
    println("최종 전역 `aw` 값: $aw")
    println("최종 전역 `bw` 값: $bw")
    if all(iszero, xwal1)
        println("경고: xwal1이 ishape=6에서도 여전히 모두 0입니다. 벽 형태가 올바르게 계산되지 않았을 수 있습니다.")
    else
        println("xwal1이 ishape=6에서 채워진 것으로 보입니다.")
    end

    # 3.6. ishape=41 테스트 (파일 및 더미 스플라인에 의존)
    println("\n--- 테스트 케이스 3: ishape = 41 (Arbitrary wall from file) ---")
    global ishape = 41
    global a = 1.0 # ishape=41의 스케일링
    global cw = 0.0 # ishape=41의 중심

    # 더미 wall_geo.in 파일 생성
    dummy_npots0 = 5
    dummy_wcentr = 0.5
    open("wall_geo.in", "w") do io
        println(io, dummy_npots0)
        println(io, dummy_wcentr)
        println(io, "theta x z") # Fortran 예시의 헤더 라인
        println(io, "0.0 1.0 0.0")
        println(io, "0.785 1.1 0.1")
        println(io, "1.57 1.2 0.5")
        println(io, "2.355 1.1 0.9")
        println(io, "3.14 1.0 1.0")
    end
    println("'wall_geo.in' 더미 파일이 ishape=41 테스트를 위해 생성되었습니다.")

    fill!(xwal1, 0.0)
    fill!(zwal1, 0.0)
    wwall!(nqnqnq_dummy, xwal1, zwal1)
    println("첫 5개 X-벽 점 (ishape 41): $(xwal1[1:min(5, end)])")
    println("첫 5개 Z-벽 점 (ishape 41): $(zwal1[1:min(5, end)])")
    println("최종 전역 `aw` 값: $aw")
    println("최종 전역 `bw` 값: $bw")
    if all(iszero, xwal1)
        println("경고: xwal1이 ishape=41에서도 여전히 모두 0입니다. 이 분기는 전체 스플라인 구현이 필요할 수 있습니다.")
    else
        println("xwal1이 ishape=41에서 채워진 것으로 보입니다 (더미 스플라인 동작 사용).")
    end
    # 더미 파일 정리
    rm("wall_geo.in")

    println("--- wwall! 테스트 완료 ---")
end

# 테스트 실행:
run_wwall_test()

--- wwall! 테스트 시작 ---

--- 테스트 케이스 1: ishape = 2 (Elliptical shell) ---
초기 플라즈마 파라미터 (xinf[1], zinf[1]): (1.3, 0.0)
전역 `a` (벽 스케일링): 1.5, `aw` (벽 오프셋): 0.2, `bw` (벽 연신율): 1.5
`ishape` = 2 테스트 중입니다.


LoadError: UndefVarError: `ishape` not defined in local scope
Suggestion: check for an assignment to a local variable that shadows a global of the same name.